In [1]:
import os
import joblib
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [3]:
data_folder = './data'
df = pd.read_csv(os.path.join(data_folder, 'prep_test.csv'))
print(df.shape)
df.head()

(1783345, 54)


,in_treineiro_Sim,nu_idade,"q001_Completou a Faculdade, mas não completou a Pós-graduação.",q001_Completou a Pós-graduação.,"q001_Completou o Ensino Médio, mas não completou a Faculdade.",q001_Nunca estudou.,q001_Não completou a 4ª série/5º ano do Ensino Fundamental.,q001_Não sei.,"q002_Completou a Faculdade, mas não completou a Pós-graduação.",q002_Completou a Pós-graduação.,"q002_Completou o Ensino Médio, mas não completou a Faculdade.",q002_Nunca estudou.,q002_Não completou a 4ª série/5º ano do Ensino Fundamental.,q002_Não sei.,"q003_Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.","q003_Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.","q003_Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.","q003_Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.",q003_Não sei.,"q004_Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.","q004_Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.","q004_Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.","q004_Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.",q004_Não sei.,q005,"q006_De R$ 1.497,01 até R$ 1.996,00.","q006_De R$ 1.996,01 até R$ 2.495,00.","q006_De R$ 11.976,01 até R$ 14.970,00.","q006_De R$ 14.970,01 até R$ 19.960,00.","q006_De R$ 2.495,01 até R$ 2.994,00.","q006_De R$ 2.994,01 até R$ 3.992,00.","q006_De R$ 3.992,01 até R$ 4.990,00.","q006_De R$ 4.990,01 até R$ 5.988,00.","q006_De R$ 5.988,01 até R$ 6.986,00.","q006_De R$ 6.986,01 até R$ 7.984,00.","q006_De R$ 7.984,01 até R$ 8.982,00.","q006_De R$ 8.982,01 até R$ 9.980,00.","q006_De R$ 9.980,01 até R$ 11.976,00.","q006_De R$ 998,01 até R$ 1.497,00.","q006_Mais de R$ 19.960,00.",q006_Nenhuma renda.,q025_Sim,tp_cor_raca_Branca,tp_cor_raca_Indígena,tp_cor_raca_Não declarado,tp_cor_raca_Preta,tp_dependencia_adm_esc_Federal,tp_dependencia_adm_esc_Privada,tp_escola_Privada,tp_

In [4]:
regressor = joblib.load('./models/2_neural_net.pkl')
regressor

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=(40, 20, 10),
                              random_state=7))])

In [5]:
y_pred = regressor.predict(df)
y_pred.shape

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
target_columns = [
    'nu_nota_lc',
    'nu_nota_ch',
    'nu_nota_cn',
    'nu_nota_mt',
    'nu_nota_redacao'
]

In [ ]:
nu_inscricao = pd.read_csv(os.path.join(data_folder, 'test.csv'), usecols=['NU_INSCRICAO'])
nu_inscricao.head()

,NU_INSCRICAO
0,5d5b362b-7388-4ac6-81b3-23573e4e2d3a
1,52356efd-3239-4cd2-a444-416625dfc560
2,1ba42e9a-dd61-4405-9b08-79b728ad23c9
3,16fa0f84-a88f-43e6-bcbb-8d5ea41e5f03
4,fa663d11-5052-4ab2-b771-3a3de3bdec55


In [ ]:
df_pred = pd.DataFrame(data=y_pred, columns=[c.upper() for c in target_columns])
df_pred.describe()

,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO
0,518.772297,499.533333,465.050422,494.291313,590.197305
1,556.503286,549.252689,509.545075,571.386389,677.688503
2,539.081847,530.109173,500.280267,561.694540,630.810812
3,547.669400,542.805456,510.160262,579.246557,620.812026
4,547.445125,542.158030,510.073585,582.613428,636.727378


In [ ]:
df_submission = nu_inscricao.join(df_pred)
df_submission.head()

,NU_INSCRICAO,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO
0,5d5b362b-7388-4ac6-81b3-23573e4e2d3a,518.772297,499.533333,465.050422,494.291313,590.197305
1,52356efd-3239-4cd2-a444-416625dfc560,556.503286,549.252689,509.545075,571.386389,677.688503
2,1ba42e9a-dd61-4405-9b08-79b728ad23c9,539.081847,530.109173,500.280267,561.694540,630.810812
3,16fa0f84-a88f-43e6-bcbb-8d5ea41e5f03,547.669400,542.805456,510.160262,579.246557,620.812026
4,fa663d11-5052-4ab2-b771-3a3de3bdec55,547.445125,542.158030,510.073585,582.613428,636.727378


In [ ]:
df_submission.to_csv('submissions/2_pred.csv', index=False)